In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shusrith/imbd-dataset-tokenized")

print("Path to dataset files:", path)

100%|██████████| 31.3M/31.3M [00:03<00:00, 10.5MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/shusrith/imbd-dataset-tokenized/versions/7


In [46]:
import pandas as pd
df = pd.read_csv("/root/.cache/kagglehub/datasets/shusrith/imbd-dataset-tokenized/versions/7/data.csv")
df

,review,sentiment
0,"[27, 4, 1, 75, 1944, 44, 1062, 11, 100, 143, 3...",1
1,"[119, 203, 3258, 68, 13, 36, 1581, 8, 12, 2198...",1
2,"[3, 384, 116, 356, 1, 1338, 2937, 6, 51, 17935...",1
3,"[9, 192, 10, 12, 3, 384, 94, 5, 1120, 57, 19, ...",1
4,"[667, 214, 3, 233, 113, 3, 116, 435, 3556, 120...",0
...,...,...
79534,"[9, 227, 3, 3510, 4325, 7, 0, 8342, 4602, 31, ...",0
79535,"[126, 109, 4, 82, 1, 1587, 4, 6509, 2263, 8903...",0
79536,"[140, 161, 5, 24, 5, 2964, 14, 1, 899, 912, 2,...",0
79537,"[700, 423, 2193, 17142, 1, 0, 11, 12204, 4212,...",0


In [47]:
import ast
df["review"] = df["review"].apply(lambda x: ast.literal_eval(x))
type(df.iloc[0, 0])

list

In [12]:
import torch.nn as nn
import torch


class RNNModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=20001)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.hidden_dim = hidden_dim

    def forward(self, x, h=None):
        x = self.embedding(x)

        if h is None:
            h = torch.zeros(1, x.size(0), self.hidden_dim).to("cuda")
        x, h = self.rnn(x, h)
        x = x[:, -1, :]
        x = self.fc(x)
        return x, h.detach()

In [13]:
model = RNNModel(64, 128, 20001)

In [34]:
import torch.nn as nn
import torch


class LSTMModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=20001)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.hidden_dim = hidden_dim

    def forward(self, x, h=None):
        x = self.embedding(x)

        if h is None:
            h = (
                torch.zeros(1, x.size(0), self.hidden_dim).to("cuda"),
                torch.zeros(1, x.size(0), self.hidden_dim).to("cuda"),
            )
        x, h = self.lstm(x, h)
        x = x[:, -1, :]
        x = self.fc(x)
        return x, (h[0].detach(), h[1].detach())

model = LSTMModel(64, 128, 20002)

In [42]:
import torch
import torch.nn as nn

class GRUModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(GRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=20001)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.hidden_dim = hidden_dim

    def forward(self, x, h=None):
        x = self.embedding(x)
        if h is None:
            h = torch.zeros(1, x.size(0), self.hidden_dim).to("cuda")
        x, h = self.gru(x, h)
        x = x[:, -1, :]
        x = self.fc(x)
        return x, h.detach()

model = GRUModel(64, 128, 20002)

In [51]:
import torch
import torch.nn as nn

class BiLSTMModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(BiLSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=20001)
        self.bilstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, 1)

    def forward(self, x, h=None):
        x = self.embedding(x)
        if h is None:
            h = (
                torch.zeros(2, x.size(0), self.bilstm.hidden_size).to("cuda"),
                torch.zeros(2, x.size(0), self.bilstm.hidden_size).to("cuda"),
            )
        x, h = self.bilstm(x, h)
        x = x[:, -1, :]
        x = self.fc(x)
        return x, h[0].detach()

model = BiLSTMModel(64, 128, 20002)

In [55]:
import torch
import torch.nn as nn

class StackedLSTMModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, num_layers):
        super(StackedLSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=20001)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim

    def forward(self, x, h=None):
        x = self.embedding(x)
        if h is None:
            h = (
                torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to("cuda"),
                torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to("cuda"),
            )
        x, h = self.lstm(x, h)
        x = x[:, -1, :]
        x = self.fc(x)
        return x, (h[0].detach(), h[1].detach())

model = StackedLSTMModel(64, 128, 20002, 2)

In [60]:
import torch
import torch.nn as nn

class StackedBiLSTMModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim1, hidden_dim2, vocab_size):
        super(StackedBiLSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=20001)

        # First BiLSTM Layer
        self.bilstm1 = nn.LSTM(embedding_dim, hidden_dim1, batch_first=True, bidirectional=True)

        # Second BiLSTM Layer (takes hidden_dim1 * 2 as input due to bidirectionality)
        self.bilstm2 = nn.LSTM(hidden_dim1 * 2, hidden_dim2, batch_first=True, bidirectional=True)

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim2 * 2, 1)

    def forward(self, x, h=None):
        x = self.embedding(x)

        # Initialize hidden states if not provided
        if h is None:
            h1 = (
                torch.zeros(2, x.size(0), self.bilstm1.hidden_size, device=x.device),
                torch.zeros(2, x.size(0), self.bilstm1.hidden_size, device=x.device),
            )
        else:
            h1 = h[:2]  # Extract first BiLSTM's hidden and cell states

        # First BiLSTM Layer
        x, h1 = self.bilstm1(x, h1)

        # Initialize hidden states for second LSTM
        if h is None:
            h2 = (
                torch.zeros(2, x.size(0), self.bilstm2.hidden_size, device=x.device),
                torch.zeros(2, x.size(0), self.bilstm2.hidden_size, device=x.device),
            )
        else:
            h2 = h[2:]  # Extract second BiLSTM's hidden and cell states

        # Second BiLSTM Layer
        x, h2 = self.bilstm2(x, h2)

        # Take the last timestep output
        x = x[:, -1, :]

        # Fully connected layer
        x = self.fc(x)

        # Return output and hidden states
        return x, (h1[0].detach(), h1[1].detach(), h2[0].detach(), h2[1].detach())

# Define model with first layer (128 units) and second layer (64 units)
model = StackedBiLSTMModel(embedding_dim=64, hidden_dim1=128, hidden_dim2=64, vocab_size=20002)
model = model.to("cuda")  # Move model to GPU


In [61]:
import torch.optim as optim

batch_size = 256
seq_len = 200
epochs = 10
learning_rate = 0.001

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [62]:
from torch.utils.data import DataLoader, TensorDataset
train = TensorDataset(
    torch.tensor(df["review"][:40000].tolist(), dtype=torch.long),
    torch.tensor(df["sentiment"][:40000].tolist(), dtype=torch.float32),
)
test = TensorDataset(
    torch.tensor(df["review"][40000:].tolist(), dtype=torch.long),
    torch.tensor(df["sentiment"][40000:].tolist(), dtype=torch.float32),
)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)

In [63]:
vocab_size = 20002
hidden_dim = 128

# for rnn, lstm, gru

In [64]:
from tqdm import tqdm

model = model.to("cuda")

for epoch in range(epochs):
    model.train()
    train_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")

    total_train_loss = 0
    correct_train = 0
    total_train = 0

    for train_data, train_labels in train_tqdm:
        train_data, train_labels = train_data.to("cuda"), train_labels.to("cuda")

        optimizer.zero_grad()
        output, _ = model(train_data)
        output = output.squeeze()

        loss = criterion(output, train_labels)
        loss.backward()
        optimizer.step()

        predictions = (torch.sigmoid(output) > 0.5).float()
        correct_train += (predictions == train_labels).sum().item()
        total_train += train_labels.size(0)

        total_train_loss += loss.item()
        train_tqdm.set_postfix(loss=loss.item())

    avg_train_loss = total_train_loss / len(train_loader)
    train_acc = correct_train / total_train

    # ------------------- Validation Loop -------------------
    model.eval()
    val_tqdm = tqdm(test_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]")

    total_val_loss = 0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for val_data, val_labels in val_tqdm:
            val_data, val_labels = val_data.to("cuda"), val_labels.to("cuda")

            output, _ = model(val_data)
            output = output.squeeze()

            loss = criterion(output, val_labels)
            total_val_loss += loss.item()

            predictions = (torch.sigmoid(output) > 0.5).float()
            correct_val += (predictions == val_labels).sum().item()
            total_val += val_labels.size(0)

            val_tqdm.set_postfix(loss=loss.item())

    avg_val_loss = total_val_loss / len(test_loader)
    val_acc = correct_val / total_val

    print(
        f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.4f}"
    )

Epoch 1/10 [Val]: 100%|██████████| 155/155 [00:03<00:00, 43.82it/s, loss=0.679]


Epoch 1/10 | Train Loss: 0.6899 | Train Acc: 0.5208 | Val Loss: 0.6744 | Val Acc: 0.5633


Epoch 2/10 [Val]: 100%|██████████| 155/155 [00:03<00:00, 42.90it/s, loss=0.725]


Epoch 2/10 | Train Loss: 0.6881 | Train Acc: 0.5381 | Val Loss: 0.6908 | Val Acc: 0.5176


Epoch 3/10 [Val]: 100%|██████████| 155/155 [00:03<00:00, 42.93it/s, loss=0.702]


...

Epoch 8/10 | Train Loss: 0.2952 | Train Acc: 0.8790 | Val Loss: 0.3971 | Val Acc: 0.8278


Epoch 9/10 [Val]: 100%|██████████| 155/155 [00:03<00:00, 43.99it/s, loss=0.441]


Epoch 9/10 | Train Loss: 0.2489 | Train Acc: 0.9019 | Val Loss: 0.3932 | Val Acc: 0.8321


Epoch 10/10 [Val]: 100%|██████████| 155/155 [00:03<00:00, 43.47it/s, loss=0.535]

Epoch 10/10 | Train Loss: 0.2117 | Train Acc: 0.9197 | Val Loss: 0.4290 | Val Acc: 0.8312
